In [1]:
import jax.numpy as jnp
from utils import apply_z_score
from kernels import RBF
from uci_datasets import Dataset


# load data
dataset = Dataset("yacht")
x_train, y_train, x_test, y_test = dataset.get_split(0)
N, D = x_train.shape
T = x_test.shape[0]
# convert to jax and apply z-score normalization
x_train, mu_x, sigma_x = apply_z_score(jnp.array(x_train))
y_train, mu_y, sigma_y = apply_z_score(jnp.array(y_train.squeeze()))
x_test = apply_z_score(jnp.array(x_test), mu=mu_x, sigma=sigma_x)
y_test = apply_z_score(jnp.array(y_test.squeeze()), mu=mu_y, sigma=sigma_y)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print(mu_x.shape, sigma_x.shape, mu_y.shape, sigma_y.shape)
# compute kernel matrix
K = RBF(x_train, x_train)
print(K.shape)


yacht dataset, N=308, d=6
(278, 6) (278,) (30, 6) (30,)
(6,) (6,) () ()
(278, 278)


In [2]:
from linear_model import exact_solution, predict
from utils import RMSE

# compute exact solution
alpha_exact = exact_solution(y_train, K)
y_pred_exact = predict(alpha_exact, x_test, x_train, RBF)
test_rmse_exact = RMSE(y_pred_exact, y_test, mu=mu_y, sigma=sigma_y)
print(test_rmse_exact)

1.0021797


Optimization with full-batch gradient (N x i, N x j and N x l)

In [3]:
import jax
import jax.random as random
import optax
from linear_model import error, regularizer
from tqdm import tqdm


alpha = jnp.zeros((N,))

learning_rate = 1e-4
momentum = .9
polyak_step_size = 1e-2
iterations = int(2 * 1e5)

optimizer = optax.sgd(learning_rate=learning_rate, momentum=momentum, nesterov=True)
opt_state = optimizer.init(alpha)


@jax.jit
def loss_fn(params):
    return error(params, y_train, K) + regularizer(params, K)


@jax.jit
def update(opt_state, params):
    loss, grad = jax.value_and_grad(loss_fn)(params)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.incremental_update(optax.apply_updates(params, updates), params, step_size=polyak_step_size)
    return opt_state, params, loss


loss_trace_full_batch = []
alpha_rmse_trace_full_batch = []
test_rmse_trace_full_batch = []

iterator = tqdm(range(iterations))
for _ in iterator:
    opt_state, alpha, loss = update(opt_state, alpha)
    y_pred = predict(alpha, x_test, x_train, RBF)

    loss_trace_full_batch.append(loss.item())
    alpha_rmse_trace_full_batch.append(RMSE(alpha_exact, alpha).item())
    test_rmse_trace_full_batch.append(RMSE(y_pred, y_test, mu=mu_y, sigma=sigma_y))


100%|██████████| 200000/200000 [00:52<00:00, 3787.09it/s]


Optimization with single data, full-batch basis, full-batch updates (1 x i, N x j and N x l)

In [4]:
import jax
import jax.random as random
import optax
from linear_model import error, regularizer
from tqdm import tqdm


key = random.PRNGKey(12345)
alpha = jnp.zeros((N,))

learning_rate = 1e-4
momentum = .9
polyak_step_size = 1e-2
iterations = int(2 * 1e5)

optimizer = optax.sgd(learning_rate=learning_rate, momentum=momentum, nesterov=True)
opt_state = optimizer.init(alpha)


@jax.jit
def loss_fn(params):
    return error(params, y_train, K) + regularizer(params, K)


def error_grad_sample(params, i):
    return -K[i, :] * (y_train[i] - K[i, :] @ params) * N


@jax.jit
def stochastic_gradient(params, i):
    return error_grad_sample(params, i) + K @ params


@jax.jit
def update(opt_state, params, key):
    loss = loss_fn(params)
    i = random.randint(key, shape=(), minval=0, maxval=N)
    grad = stochastic_gradient(params, i)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.incremental_update(optax.apply_updates(params, updates), params, step_size=polyak_step_size)
    return opt_state, params, loss


idx = jnp.arange(start=0, stop=N)
grad_samples = jax.vmap(stochastic_gradient, (None, 0))(random.normal(random.PRNGKey(123), (N,)), idx)
grad_mean = grad_samples.sum(axis=0) / N
grad_cov = (grad_samples.T @ grad_samples) / N - jnp.outer(grad_mean, grad_mean)
grad_cov_trace = jnp.trace(grad_cov) / N
print(f"cov trace: {grad_cov_trace}")


loss_trace_1xi_Nxj_Nxl = []
alpha_rmse_trace_1xi_Nxj_Nxl = []
test_rmse_trace_1xi_Nxj_Nxl = []

iterator = tqdm(range(iterations))
for _ in iterator:
    key, subkey = random.split(key)
    opt_state, alpha, loss = update(opt_state, alpha, subkey)
    y_pred = predict(alpha, x_test, x_train, RBF)

    loss_trace_1xi_Nxj_Nxl.append(loss.item())
    alpha_rmse_trace_1xi_Nxj_Nxl.append(RMSE(alpha_exact, alpha).item())
    test_rmse_trace_1xi_Nxj_Nxl.append(RMSE(y_pred, y_test, mu=mu_y, sigma=sigma_y))

cov trace: 13761.0654296875


100%|██████████| 200000/200000 [01:01<00:00, 3234.18it/s]


Optimization with single data, single basis, full-batch updates (1 x i, 1 x j and N x l), high lr

In [5]:
import jax
import jax.random as random
import optax
from linear_model import error, regularizer
from tqdm import tqdm


key = random.PRNGKey(12345)
alpha = jnp.zeros((N,))

learning_rate = 1e-4
momentum = .9
polyak_step_size = 1e-2
iterations = int(2 * 1e5)

optimizer = optax.sgd(learning_rate=learning_rate, momentum=momentum, nesterov=True)
opt_state = optimizer.init(alpha)


@jax.jit
def loss_fn(params):
    return error(params, y_train, K) + regularizer(params, K)


def error_grad_sample(params, i, j):
    return -K[i, :] * (y_train[i] - (K[i, j] * params[j]) * N) * N


@jax.jit
def stochastic_gradient(params, i, j):
    return error_grad_sample(params, i, j) + K @ params


@jax.jit
def update(opt_state, params, key):
    loss = loss_fn(params)
    i, j = random.randint(key, shape=(2,), minval=0, maxval=N)
    grad = stochastic_gradient(params, i, j)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.incremental_update(optax.apply_updates(params, updates), params, step_size=polyak_step_size)
    return opt_state, params, loss


idx = jnp.arange(start=0, stop=N)
grad_samples = jax.vmap(jax.vmap(stochastic_gradient, (None, None, 0)), (None, 0, None))(random.normal(random.PRNGKey(123), (N,)), idx, idx).reshape(N ** 2, N)

grad_mean = grad_samples.sum(axis=0) / (N ** 2)
grad_cov = (grad_samples.T @ grad_samples) / (N ** 2) - jnp.outer(grad_mean, grad_mean)
grad_cov_trace = jnp.trace(grad_cov) / N
print(f"cov trace: {grad_cov_trace}")


loss_trace_1xi_1xj_Nxl_high_lr = []
alpha_rmse_trace_1xi_1xj_Nxl_high_lr = []
test_rmse_trace_1xi_1xj_Nxl_high_lr = []

iterator = tqdm(range(iterations))
for _ in iterator:
    key, subkey = random.split(key)
    opt_state, alpha, loss = update(opt_state, alpha, subkey)
    y_pred = predict(alpha, x_test, x_train, RBF)

    loss_trace_1xi_1xj_Nxl_high_lr.append(loss.item())
    alpha_rmse_trace_1xi_1xj_Nxl_high_lr.append(RMSE(alpha_exact, alpha).item())
    test_rmse_trace_1xi_1xj_Nxl_high_lr.append(RMSE(y_pred, y_test, mu=mu_y, sigma=sigma_y))

cov trace: 4223077.0


100%|██████████| 200000/200000 [00:55<00:00, 3623.64it/s]


Optimization with single data, single basis, full-batch updates (1 x i, 1 x j and N x l), low lr

In [6]:
import jax
import jax.random as random
import optax
from linear_model import error, regularizer
from tqdm import tqdm


key = random.PRNGKey(12345)
alpha = jnp.zeros((N,))

learning_rate = 1e-5
momentum = .9
polyak_step_size = 1e-2
iterations = int(2 * 1e5)

optimizer = optax.sgd(learning_rate=learning_rate, momentum=momentum, nesterov=True)
opt_state = optimizer.init(alpha)


@jax.jit
def loss_fn(params):
    return error(params, y_train, K) + regularizer(params, K)


def error_grad_sample(params, i, j):
    return -K[i, :] * (y_train[i] - (K[i, j] * params[j]) * N) * N


@jax.jit
def stochastic_gradient(params, i, j):
    return error_grad_sample(params, i, j) + K @ params


@jax.jit
def update(opt_state, params, key):
    loss = loss_fn(params)
    i, j = random.randint(key, shape=(2,), minval=0, maxval=N)
    grad = stochastic_gradient(params, i, j)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.incremental_update(optax.apply_updates(params, updates), params, step_size=polyak_step_size)
    return opt_state, params, loss


loss_trace_1xi_1xj_Nxl_low_lr = []
alpha_rmse_trace_1xi_1xj_Nxl_low_lr = []
test_rmse_trace_1xi_1xj_Nxl_low_lr = []

iterator = tqdm(range(iterations))
for _ in iterator:
    key, subkey = random.split(key)
    opt_state, alpha, loss = update(opt_state, alpha, subkey)
    y_pred = predict(alpha, x_test, x_train, RBF)

    loss_trace_1xi_1xj_Nxl_low_lr.append(loss.item())
    alpha_rmse_trace_1xi_1xj_Nxl_low_lr.append(RMSE(alpha_exact, alpha).item())
    test_rmse_trace_1xi_1xj_Nxl_low_lr.append(RMSE(y_pred, y_test, mu=mu_y, sigma=sigma_y))

100%|██████████| 200000/200000 [00:56<00:00, 3549.26it/s]


In [7]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize=[15, 5])
ax1 = fig.add_subplot(1, 3, 1)
ax1.plot(loss_trace_full_batch, color='k', label='Nxi, Nxj, Nxl, lr=1e-4')
ax1.plot(loss_trace_1xi_Nxj_Nxl, color='tab:blue', label='1xi, Nxj, Nxl, lr=1e-4')
ax1.plot(loss_trace_1xi_1xj_Nxl_high_lr, color='tab:orange', label='1xi, 1xj, Nxl, lr=1e-4')
ax1.plot(loss_trace_1xi_1xj_Nxl_low_lr, color='tab:green', label='1xi, 1xj, Nxl, lr=1e-5')
ax1.axhline(loss_fn(alpha_exact), color='k', linestyle='--', label='Exact')
ax1.set_xlabel("Iterations")
ax1.set_ylabel("Loss")
ax1.semilogy()
ax1.grid(alpha=.5)
ax1.legend()

ax2 = fig.add_subplot(1, 3, 2)
ax2.plot(alpha_rmse_trace_full_batch, color='k', label='Nxi, Nxj, Nxl, lr=1e-4')
ax2.plot(alpha_rmse_trace_1xi_Nxj_Nxl, color='tab:blue', label='1xi, Nxj, Nxl, lr=1e-4')
ax2.plot(alpha_rmse_trace_1xi_1xj_Nxl_high_lr, color='tab:orange', label='1xi, 1xj, Nxl, lr=1e-4')
ax2.plot(alpha_rmse_trace_1xi_1xj_Nxl_low_lr, color='tab:green', label='1xi, 1xj, Nxl, lr=1e-5')
ax2.axhline(0., color='k', linestyle='--', label='Exact')
ax2.set_xlabel("Iterations")
ax2.set_ylabel(r"$\alpha$ RMSE")
ax2.grid(alpha=.5)
ax2.legend()

ax3 = fig.add_subplot(1, 3, 3)
ax3.plot(test_rmse_trace_full_batch, color='k', label='Nxi, Nxj, Nxl, lr=1e-4')
ax3.plot(test_rmse_trace_1xi_Nxj_Nxl, color='tab:blue', label='1xi, Nxj, Nxl, lr=1e-4')
ax3.plot(test_rmse_trace_1xi_1xj_Nxl_high_lr, color='tab:orange', label='1xi, 1xj, Nxl, lr=1e-4')
ax3.plot(test_rmse_trace_1xi_1xj_Nxl_low_lr, color='tab:green', label='1xi, 1xj, Nxl, lr=1e-5')
ax3.axhline(test_rmse_exact, color='k', linestyle='--', label='Exact')
ax3.set_xlabel("Iterations")
ax3.set_ylabel("Test RMSE")
ax3.semilogy()
ax3.grid(alpha=.5)
ax3.legend()

fig.tight_layout()